<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cpp_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
functional programming in c++ をなぞり読み

https://www.manning.com/books/functional-programming-in-c-plus-plus#toc

In [ ]:
# 最初の例
# 命令型
# Listing 1.1 Calculating the number of lines the imperative way
%%script false
vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

In [25]:
# 動く形にして見よう
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

int main() {
    ofstream ofs01("./temp01.txt");
    ofs01 << "foo bar baz \nThis is a pen" << endl;
    ofs01.close(); // 2行
    ofstream ofs02("./temp02.txt");
    ofs02 << "日本語もどうか" << endl << endl;
    ofs02 << "これが最後" << endl << "ん" << endl; 
    ofs02.close(); // 4行

    vector<string> files = {"temp01.txt", "temp02.txt"};
    vector<int> lines;
    lines = count_lines_in_files(files);
    for(int i : lines) {
        cout << i << endl;
    }
}

Overwriting temp.cpp


In [26]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# とりあえず動いた
# "\n" を数えているのでファイル末に改行がないと正しい数字にならない
# getline を使いたい
# テキストはこれを関数型に変えると言うことだがどうするのだろうか

In [29]:
# Listing 1.2 Using std::count to count newline characters
# count version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  ifstream in(filename);
  return count(istreambuf_iterator<char>(in), istreambuf_iterator<char>(), '\n');
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [30]:
!g++ temp.cpp; ./a.out

2
4


In [31]:
# count version は count を使っている事より、istreambuf_iterator を知らなかった
# cpprefjp による例
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int main()
{
  stringstream ss;
  ss << "1 2 3" << endl
     << "4 5 6";

  // 文字列の入力ストリームから順に文字を読み込むイテレータを用意
  istreambuf_iterator<char> it(ss);
  istreambuf_iterator<char> last;

  // イテレータを進めることにより、入力ストリームからデータを順に読み取る
  for_each(it, last, [](char c) { std::cout << c; });
}

Overwriting temp.cpp


In [32]:
!g++ temp.cpp; ./a.out

1 2 3
4 5 6

In [35]:
# ちょっとよくわかりません
# とりあえず getline version を作って、ファイル末の改行なしに対応するか実験しよう #=> 成功!!!!
# getline version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [36]:
!g++ temp.cpp; ./a.out

2
4


In [41]:
# テキストは次に transform を使っている
# transform は他の言語では map に当たるアルゴリズムである
# map -> transform, reduce -> accumulate, filter -> copy_if らしい
# transform version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<std::string>& files) {
    vector<int> results(files.size());
    // transform(files.cbegin(), files.cend(), results.begin(), count_lines);    
    transform(files.begin(), files.end(), results.begin(), count_lines); // cbegin と begin の違いは const かどうかでほとんど同じ
    return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [42]:
!g++ temp.cpp; ./a.out

2
4


In [43]:
# ranges version
# ranges は chapter 7 で説明するとのこと
# だがエラーなのでとりあえず %%script false にして先へ進む
%%script false
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
    return files | transform(count_lines);
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

In [ ]:
# 最終形は次のようになる、とのこと
# transform を 2回に分けている
%%script false
vector<int>
count_lines_in_files(const vector<string>& files) {
    return files | transform(open_file) | transform(count_lines);
}